# Using the NGD API with Python

The National Geographic Database (NGD) is Ordnance Survey's next generation API that makes it easier to use our data.

There are many ways to access the NGD. If you'd like to download data using a GUI, OS has [Select+Build](https://www.ordnancesurvey.co.uk/business-government/products/os-select-build), an intuitive website that allows you to download multiple types of features at once. There is also a [REST API](https://www.ordnancesurvey.co.uk/business-government/products/os-ngd-api-features) for programatically access features by querying endpoints. If you are a JavaScript developer, we even created the [`osdatahub JavaScript library`](https://github.com/OrdnanceSurvey/osdatahub-js) that makes it easy to use NGD with Node.js or through the browser.

For Python developers, we have the [`osdatahub Python library`](https://github.com/OrdnanceSurvey/osdatahub). The `osdatahub` package wraps almost all APIs in the [OS Data Hub](https://osdatahub.os.uk/) and provides an easy, Pythonic way to get data from Ordnance Survey. We have now added support for the NGD Features API, so you can access OS' newest product with only a few lines of Python code.

This notebook takes us through using the NGD with the `osdatahub` Python library.

## Setup

In order to access the NGD, you  must first have an OS Data Hub api key. [This Python notebook](https://github.com/OrdnanceSurvey/osdatahub/blob/master/Examples/Setting%20up%20an%20API%20key.ipynb) takes you through registering an account and getting an API key. When you're choosing which APIs you want to add to your project, you must select "OS NGD API - Features".

**Note:** The NGD API is a premium or public sector product only. If you are looking to access free data, you should use the [OS Features API](https://osdatahub.os.uk/docs/wfs/overview).

Once you have an API key, you can load your API key by passing it in as a string, or setting it as an environment variable. We recommend using the [`python-dotenv`](https://pypi.org/project/python-dotenv/) library to securely store your API keys without risk of accidentally committing them in your repo.

To use the `python-dotenv` package, simply:
1. Install with pip: `pip install python-dotenv`
2. Create a `.env` file in the root directory of your project
3. Add your API key to the file: `OS_API_KEY=[YOUR API KEY HERE]`


In [15]:
# setting api key as a string
key = "YOUR API KEY HERE"

# OR, using python-dotenv
from dotenv import load_dotenv
from os import environ

load_dotenv()

key = environ.get("OS_API_KEY")
key

'jObazFUHCQgodV58uKS0fURLtTLS2inw'

## Choosing a Collection

The NGD can provide many types of feature, such as rail lines, roads, buildings, and waterways.

To find the full list, you can either look at [the API's Technical Specification](https://osdatahub.os.uk/docs/ofa/technicalSpecification), or you can query using Python.

We start by importing the NGD:

In [5]:
from osdatahub import NGD

A full, up-to-date list of available collections can be gotten through the `get_collections()` class method:

In [6]:
NGD.get_collections()

{'links': [{'href': 'https://api.os.uk/features/ngd/ofa/v1/collections',
   'rel': 'self',
   'type': 'application/json',
   'title': 'All collections'}],
 'collections': [{'id': 'bld-fts-buildingline',
   'title': 'Building Line',
   'description': 'A feature which has a line geometry and represents a wall between two buildings, building internal divisions, or an overhanging building edge.',
   'crs': ['http://www.opengis.net/def/crs/EPSG/0/27700',
    'http://www.opengis.net/def/crs/EPSG/0/3857',
    'http://www.opengis.net/def/crs/EPSG/0/4326',
    'http://www.opengis.net/def/crs/OGC/1.3/CRS84'],
   'storageCrs': 'http://www.opengis.net/def/crs/EPSG/0/27700',
   'itemType': 'feature',
   'extent': {'spatial': {'bbox': [[-8.82, 49.79, 1.92, 60.94]],
     'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'},
    'temporal': {'interval': [['2022-08-27T00:00:00Z', None]],
     'trs': 'http://www.opengis.net/def/uom/ISO-8601/0/Gregorian'}},
   'links': [{'href': 'https://api.os.uk/feat

After looking at the available collections, we'll be using the Building Part feature. The NGD API uses the id for a given collection, which in our case is `bld-fts-buildingpart`. You can find out more about how the ids are created by reading the Content section of the [Technical Specification](https://osdatahub.os.uk/docs/ofa/technicalSpecification)

## Getting your First Features

Now that we have our API key and the id of the collection we'd like to get, we are ready to go! To get the first 100 features in the collection, you simply need to make an `NGD` object and then run the `query()` method:

In [8]:
collection = "bld-fts-buildingpart"

ngd = NGD(key, collection)
features = ngd.query()
features

{'type': 'FeatureCollection',
 'links': [{'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?limit=100',
   'rel': 'self',
   'type': 'application/geo+json',
   'title': "All features from the 'Building Part' collection"},
  {'href': 'https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items?offset=100&limit=100',
   'rel': 'next',
   'type': 'application/geo+json',
   'title': 'Next page'}],
 'timeStamp': '2022-11-04T12:03:43.220253Z',
 'numberReturned': 100,
 'features': [{'id': '000000b1-0556-4231-a52a-9b5b8b82dfbf',
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-3.9703075, 55.7425473],
      [-3.9701862, 55.7424916],
      [-3.9701342, 55.7425277],
      [-3.9702899, 55.7425992],
      [-3.9703201, 55.7425783],
      [-3.9702857, 55.7425625],
      [-3.9703075, 55.7425473]]]},
   'properties': {'osid': '000000b1-0556-4231-a52a-9b5b8b82dfbf',
    'toid': 'osgb1000041024621',
    'theme': 'Building

The API returns an OGC-compliant GeoJSON, easy for importing into other libraries for analysis. For example, in order to import into `geopandas`:

In [13]:
import geopandas as gpd

gdf = gpd.GeoDataFrame.from_features(features)
gdf.head()

,geometry,osid,toid,theme,changetype,isobscured,description,versiondate,geometry_area,height_source,...,absoluteheightroofbase,description_updatedate,oslandcover_updatedate,oslanduse_evidencedate,relativeheightroofbase,versionavailabletodate,firstdigitalcapturedate,description_evidencedate,oslandcover_evidencedate,versionavailablefromdate
0,"POLYGON ((-3.97031 55.74255, -3.97019 55.74249...",000000b1-0556-4231-a52a-9b5b8b82dfbf,osgb1000041024621,Buildings,Modified Attributes,False,Building,2022-09-18,59.200650,Ordnance Survey,...,110.85,2006-08-30,2006-08-30,2006-08-30,5.85,None,1991-09-18,2006-08-30,2006-08-30,2022-09-19T00:00:00Z
1,"POLYGON ((1.72756 52.66142, 1.72755 52.66144, ...",00000183-3ae0-4f05-adb8-a5792d09f55f,osgb5000005167584940,Buildings,New,False,Building,2022-08-26,9.499142,Ordnance Survey,...,6.21,2015-10-29,2015-10-29,2015-03-11,0.82,None,2015-11-12,2015-03-11,2015-03-11,2022-08-27T00:00:00Z
2,"POLYGON ((0.19415 51.59241, 0.19411 51.59241, ...",000001d6-8217-4f7a-a70e-3757eb76c9e4,osgb1000000360448,Buildings,New,False,Building,2022-08-26,15.258750,Ordnance Survey,...,37.13,1993-04-01,1993-04-01,1993-04-01,2.43,None,1993-04-01,1993-04-01,1993-04-01,2022-08-27T00:00:00Z
3,"POLYGON ((-1.45472 52.39610, -1.45479 52.39613...",00000208-a659-49f9-a587-ab26cb2b2248,osgb1000017249389,Buildings,New,False,Building,2022-08-26,12.806250,Ordnance Survey,...,71.95,1993-05-01,1993-05-01,1993-05-01,1.96,None,1993-05-01,1993-05-01,1993-05-01,2022-08-27T00:00:00Z
4,"POLYGON ((0.93898 52.81772, 0.93900 52.81767, ...",00000246-42c8-4e23-982e-15613e0fd2fe,osgb1000008341371,Buildings,New,False,Building,2022-08-26,58.280086,Ordnance Survey,...,54.46,2005-04-30,2005-04-30,2005-04-30,3.61,None,1992-03-13,2005-04-30,2005-04-30,2022-08-27T00:00:00Z


You can learn more about importing data from the `osdatahub` package into other common Python libraries [in this example](https://github.com/OrdnanceSurvey/osdatahub/blob/master/Examples/Plotting%20API%20Results%20-%20GeoPandas%2C%20Matplotlib%20and%20Contextily.ipynb).

## Adding Filters

There are many options for filtering your NGD query, using the following attributes:

* `extent` - you can specify any polygon to query by using the `Extent` class. You can learn more about using `Extent` [here](https://github.com/OrdnanceSurvey/osdatahub/blob/master/Examples/Defining%20Extents%20for%20API%20Queries.ipynb)
* `start_datetime` and `end_datetime` - if you want to only get features that have a temporal property, you can specify date ranges to query within. If you want to get features for a single time, simply provide the same argument for both parameters.
* `cql_filter` - The NGD API supports a generic filter grammar called the Common Query Language (CQL) to further filter your query using human readable commands. You can find out more about the operations that the API supports in the Queryables section of the [Technical Specification](https://osdatahub.os.uk/docs/ofa/technicalSpecification). The CQL filter allows you to specify specific properties for features as well as spatial filters.
* `max_results` - Allows you to specify the maximum number of features you'd like to receive.
* `offset` - Skips past the specified number of features in the collection.

In our case, we want to get features that are only within a certain bounding box in Manchester. We can do this by specifying an extent:

In [14]:
from osdatahub import Extent

extent = Extent.from_bbox((-2.244973,53.476620,-2.237799,53.480525), crs="EPSG:4326")
features = ngd.query(extent=extent)
features

ERROR:root:{
    "code": 400,
    "description": "Intersection geometry 'POLYGON ((-2.237799 53.47662, -2.237799 53.480525, -2.244973 53.480525, -2.244973 53.47662, -2.237799 53.47662))' is outside the spatial extent of collection 'Building Part'. If the given geometry is within a supported CRS, please use the filter-crs parameter. The following coordinate reference systems are supported: http://www.opengis.net/def/crs/OGC/1.3/CRS84, http://www.opengis.net/def/crs/EPSG/0/27700, http://www.opengis.net/def/crs/EPSG/0/4326, http://www.opengis.net/def/crs/EPSG/0/3857",
    "help": "https://osdatahub.os.uk/docs/ofa/technicalSpecification"
}


HTTPError: 400 Client Error: Bad Request for url: https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-buildingpart/items/?filter=INTERSECTS%28geometry%2C+POLYGON+%28%28-2.237799+53.47662%2C+-2.237799+53.480525%2C+-2.244973+53.480525%2C+-2.244973+53.47662%2C+-2.237799+53.47662%29%29%29&filter-crs=http%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FEPSG%2F0%2F4326&limit=100&offset=0

In [ ]:
## Basically i was just gonna add more filters, and talk about those. Intersections, choosing by properties etc.